<br/>
<br/>
<br/>
<br/>
<center><b><span style="font-size:2.7em"> Bancor and UniSwap - A Closer Look <br/> <br/>
    <br/> <br/> </span></b></center> 

<img src="https://3mgj4y44nc15fnv8d303d8zb-wpengine.netdna-ssl.com/wp-content/uploads/2018/06/celo-696x449.jpg" align="center" style="width:50%"/>
<center><a href="roman@celo.org">roman@celo.org</a> / <a href="roman@celo.org">markus@celo.org</a></center>
<br/>
<br/>
<br/>
<br/>
<br/>


In [1]:
# import modules and set up the notebook
import warnings

import numpy as np
import pandas as pd
import plotly
from IPython.display import display, HTML
from plots import plotly_from_df, plotly_stacked_from_df
from ipywidgets import IntProgress
from bancor import *
from stat_utils import gbm
# makes offline plotly plots appear in notebook
plotly.offline.init_notebook_mode(connected=True)

# Bancor

## Theoretical Background

### Path independence

A [blog post of Vitalik](https://vitalik.ca/general/2017/06/22/marketmakers.html) discusses the path independence property of certain on-chain market makers. He however does not define this property explicitely nor does he explicitely state whether the Bancor system actually satisfies it but only gives some numerical example of a certain on-chain market maker setup which satisfies it. 

Hence the question:
* What is path idependence in a Bancor-type system exactly?
* Does the Bancor setup satisfy the path indepence property - and if so, why?

**A path independence definition:**
Let's define that a Bancor-type system is path independent if and only if the smart token supply $S_t$ and the connector balance $C_t$ are independent of the set of all prices $P_q$ with $0<q<t$ given a price $P_t$. 

Is the Bancor system as described in the Bancor white paper path independent? An [add-on to the white paper](https://drive.google.com/file/d/0B3HPNP-GDn7aRkVaV3dkVl9NS2M/view) shows that 

$P_t=\left(\frac{S_t}{S_0}\right)^{(1-F)/F} P_0$

where $F$ is the connector weight. Rearranging gives

$S_t = \left(\frac{P_t}{P_0}\right)^{\frac{F}{1-F}} S_0$.

Because $F$ is just a known constant, it can be seen that **$S_t$ is independent of the set of all prices $P_q$ with $0<q<t$ given a price $P_t$**.

Using the definition of the connector weight $F$ (see white paper) 

$F = \frac{C_t}{S_t P_t}$

and the independence of $S_t$ established above, it directly shows that the future connector balance **$C_t$ is also independent of the set of all prices $P_q$ with $0<q<t$ given a price $P_t$**. 

Therefore, the **Bancor system as described in the white paper is path independent**. However, slight changes like using time varying connector weight that depends on some state variable $x_t$, i.e. $F\!\left(x_t\right)$, can be expected to remove this property. 

**Note: Introducing a bid-ask type spread will also remove the path-independence property!**

**Note: Given an initial target price, we have two degrees of freedom as can be seen from $F = \frac{C_t}{S_t P_t}$.**


### Sensitivities

The sensitivities of the balances with respect to changes in price are not derived in the Bancor white paper. Both sensitivies, and their dependence on the conector weight and, are hower important to understand to be able to select a appropriate connector weight. 

Using 

$S_t = \left(\frac{P_t}{P_0}\right)^{\frac{F}{1-F}} S_0$

from the last section one can derive that a percentage price change of $dP/P$ leads to a percentage smart token supply change $dS/S$ of 

$dS/S = \left(1+dP/P\right)^{F/\left(1-F\right)}$.

Similarly, one can derive that a percentage price change of $dP/P$ leads to a percentage connector balance change $dC/C$ of

$dC/C= \left(1+dP/P\right)^{1/\left(1-F\right)}-1$.

The nice thing about the connector function used by Bancor is that it generates constant sensitivities as can be seen from above.

In [2]:
# plot sensitivities (assuming price change of 1%)
c_weight_lower = 0.01  # must be bigger 0
c_weight_upper = 0.99  # must be smaller 1
grid_step_size = 0.01
n_steps = int(round((c_weight_upper - c_weight_lower + grid_step_size)/grid_step_size))
c_weight_grid = np.linspace(c_weight_lower, c_weight_upper, n_steps)

s_sens = []
c_sens = []
for c_weight in c_weight_grid:
    s_sens.append(get_s_sensitivity(c_weight))
    c_sens.append(get_c_sensitivity(c_weight))
sens_df = pd.DataFrame([s_sens, c_sens], index=['s_sens', 'c_sens'], columns=c_weight_grid).T

iplot_dict = plotly_from_df(sens_df, title='Sensitivities: Pct. Change in Balances given Price Increase of 1%', xlabel='c_weight', y_axis_type='log', mode='lines')
plotly.offline.iplot(iplot_dict)

## Checks and Numerical Examples

In [3]:
# Run numerical examples from section 3.4 of bancor WP to test functions
s_supply = 1000
c_balance = 250
c_weight = 0.5
delta_c = 10

s_price = get_s_price(c_balance, c_weight, s_supply)
print('price = ' + str(s_price))

delta_s = get_delta_s(delta_c, c_balance, c_weight, s_supply)
print('tokens issued = ' + str(delta_s))

s_effective_price = get_effective_price(delta_c, delta_s)
print('effective price = ' + str(s_effective_price))



price = 0.5
tokens issued = 19.803902718557033
effective price = 0.5049509756796375


In [4]:
# Check delta_s calculation for some arbitrary target price
target_price = 0.6
delta_s = get_delta_s_from_price(target_price, c_balance, c_weight, s_supply)
delta_c = get_delta_c(delta_s, c_balance, c_weight, s_supply)
new_s_supply = s_supply + delta_s
new_c_balance = c_balance + delta_c
new_price = get_s_price(new_c_balance, c_weight, new_s_supply)
print('Target price was ' + str(target_price) + ' and new price after issuing an additional ' + str(delta_s) + ' smart tokens is ' + str(new_price))

Target price was 0.6 and new price after issuing an additional 200.0 smart tokens is 0.6


## Bancor vs. Centralized Exchange (Most Simple Setup)

A smart_toke-to-connector_token exchange rate on a centralized exchange is simulated via a Geometric Brownian Motion. It is then assumed that an arbitrageur eleminates all price differences in the Bancor system immediately. Spreads on the centralized exchange and in the Bancor system are assumed to be zero. The liquidity on the centralized exchange is assumed to be infinite (no slippage). 

In [5]:
# Set parameters
# Bancor initial setup (assumes that c_balance is chosen as such that everything else works out)
s_supply = 1000
s_price = 0.5  # given in connector units
c_weight = 0.5  # 0 < c_weight < 1 (bancor formulas actually rule out 0 and 1!)
c_balance = get_c_balance(s_price, c_weight, s_supply)

# GBM parameters
x0 = s_price # start fx price at current bancor price (assumes inital bancor setup was smartly chosen)
mu = 0
sigma = 1
dt = 1/360  # one step per day
steps = int(1/dt)
sim_iter = 1
seed = 2  # 0 for random seed, otherwise fixed seed

### One Path in Detail

In [24]:
# Simulate an exchange rate at centralized exchange and arbitrage away difference to bancor

# This gives a Smart_token-to-Connector_Token exchange rate time series
fx_rate_exchange = gbm(x0=x0, mu=mu, cov=sigma, dt=dt, steps=steps, sim_iter=sim_iter, randomize=seed)

# Initialize
delta_s = [np.nan]
delta_c = [np.nan]
delta_c_fx = [np.nan]
bancor_price = [s_price]
current_s_supply = s_supply
current_c_balance = c_balance

# day 0 is the initial setting before the bancor system goes live
# arbitrage starts at day 1
for current_rate in fx_rate_exchange[0][1:]:
    
    # get Celo Dollar supply change to achieve target price
    delta_s.append(get_delta_s_from_price(current_rate, current_c_balance, c_weight, current_s_supply))
    # get corresponding change in gold balance
    delta_c.append(get_delta_c(delta_s[-1], current_c_balance, c_weight, current_s_supply))

    # adjust bancor balances
    current_s_supply += delta_s[-1]
    current_c_balance += delta_c[-1]
    
    # calculate change to gold holdings if same amount expansion/contraction of smart toke would happen at fx rate
    delta_c_fx.append(delta_s[-1] * current_rate)

    # get bancor price
    bancor_price.append(get_s_price(current_c_balance, c_weight, current_s_supply))

# collect results in dataframes
results_df = pd.DataFrame()
results_df['bancor_price'] = bancor_price
results_df['fx_rate_exchange'] = fx_rate_exchange[0]
results_df['delta_s'] = delta_s
results_df['delta_c'] = delta_c
results_df['c_balance_fx'] = np.nancumsum(delta_c_fx) + c_balance
results_df['s_supply'] = np.nancumsum(delta_s) + s_supply
results_df['c_balance'] = np.nancumsum(delta_c) + c_balance
results_df.index.name = 'step'

In [7]:
# Show both exchange rates after arbitrage trades
iplot_dict = plotly_from_df(results_df[['bancor_price', 'fx_rate_exchange']], title='Exchange Rates', xlabel='step')
plotly.offline.iplot(iplot_dict)

In [8]:
# Show difference in exchange rates after Bancor arbitrage trades
results_df['bancor_minus_fx'] = results_df['bancor_price'] - results_df['fx_rate_exchange']
iplot_dict = plotly_from_df(results_df['bancor_minus_fx'], title='Bancor Price Minus FX-Rate', xlabel='step')
plotly.offline.iplot(iplot_dict)

In [9]:
# s_supply and c_balance
results_df['contracted_s_tokens'] = -np.nancumsum(results_df['delta_s'])
results_df['contracted_s_tokens_value'] = results_df['contracted_s_tokens'] * results_df['fx_rate_exchange']
results_df['connector_value'] = results_df['c_balance']
iplot_dict = plotly_from_df(results_df[['s_supply', 'c_balance', 'c_balance_fx']], title='Coin Balances', xlabel='step')
plotly.offline.iplot(iplot_dict)

In the above figure, c_balance_fx gives the connector balance if the smart toke supply would have been adjusted as in the Bancor case but by paying the fx rate of the centralized exchange without slippage.

In [10]:
# Show value of bancor market maker vs centralized exchange total value
results_df['bancor_market_maker_value'] = results_df['contracted_s_tokens_value'] + results_df['c_balance']
results_df['fx_market_maker_value'] = results_df['contracted_s_tokens_value'] + results_df['c_balance_fx']

iplot_dict = plotly_from_df(results_df[['bancor_market_maker_value', 'fx_market_maker_value']], title='Total Market Maker Value', xlabel='step')
plotly.offline.iplot(iplot_dict)

The market maker values are given in units of the connector token. For both market maker types, the market maker value at each point in time is the connector balance plus the value of the smart tokens that have been taken out of circulation (negative if smart toke supply has expanded overall).

In [11]:
# Arbitrage loss of bancor market maker compared to no slippage trades on centralized exchange
results_df['bancor_slippage_loss'] = results_df['fx_market_maker_value'] - results_df['bancor_market_maker_value']

iplot_dict = plotly_from_df(results_df['bancor_slippage_loss'], title='Bancor Arbitrage Loss (in Connector Units)', xlabel='step')
plotly.offline.iplot(iplot_dict)

This figure shows how much the bancor system looses vs. a system in which the same smart token supply changes are achieved by buying/seeling at the centralized exchange without any slippage. It is computed as the difference of the two market maker values shown in the previous graph.

### Average Arb-Losses over Grid of Connector Weights

In [12]:
# Set bancor parameters
# Bancor initial setup (assumes that c_balance is chosen as such that everything else works out)
s_supply = 1000
s_price = 1  # given in connector units
c_balance = get_c_balance(s_price, c_weight, s_supply)

# GBM parameters
x0 = s_price # start fx price at current bancor price (assumes inital bancor setup was smartly chosen)
mu = 0
sigma = 1
dt = 1/360  # one step per day
steps = int(1/dt)  # one year overall
seed = 2  # 0 for random seed, otherwise fixed seed for every grid iteration

In [13]:
# set parameters for grid run
sim_iter = 25  # number of iterations per connector weight
c_weight_lower = 0.1  # must be bigger 0
c_weight_upper = 0.9  # must be samller 1
grid_step_size = 0.01

In [14]:
# Simulate over grid of c_weights and show average total arbitrage losses
n_steps = int(round((c_weight_upper - c_weight_lower + grid_step_size)/grid_step_size))
c_weight_grid = np.linspace(c_weight_lower, c_weight_upper, n_steps)

results_list = []
c_weight_arb_loss = dict()

grid_run_progress = IntProgress(min=0, max=len(c_weight_grid), description="{:.1%}".format(0)) # instantiate progress bar
display(grid_run_progress) # display the bar

for c_weight in c_weight_grid:
       
    c_balance = get_c_balance(s_price, c_weight, s_supply)  # follows from c_weight and other parameters
    
    # This gives a Smart_token-to-Connector_Token exchange rate time series
    fx_rate_exchange = gbm(x0=x0, mu=mu, cov=sigma, dt=dt, steps=steps, sim_iter=sim_iter, randomize=seed)

    # day 0 is the initial setting before the bancor system goes live
    # arbitrage starts at day 1
    total_arb_losses = []
    for this_fx_rate_exchange in fx_rate_exchange:
        
        # Initialize
        delta_s = [np.nan]
        delta_c = [np.nan]
        delta_c_fx = [np.nan]
        bancor_price = [s_price]
        current_s_supply = s_supply
        current_c_balance = c_balance
        
        for current_rate in this_fx_rate_exchange[1:]:

            # get Celo Dollar supply change to achieve target price
            delta_s.append(get_delta_s_from_price(current_rate, current_c_balance, c_weight, current_s_supply))
            # get corresponding change in gold balance
            delta_c.append(get_delta_c(delta_s[-1], current_c_balance, c_weight, current_s_supply))

            # adjust bancor balances
            current_s_supply += delta_s[-1]
            current_c_balance += delta_c[-1]

            # calculate change to gold holdings if same amount expansion/contraction of smart toke would happen at fx rate
            delta_c_fx.append(delta_s[-1] * current_rate)

            # get bancor price
            bancor_price.append(get_s_price(current_c_balance, c_weight, current_s_supply))

        # collect results in dataframes
        results_df = pd.DataFrame()
        results_df['bancor_price'] = bancor_price
        results_df['fx_rate_exchange'] = fx_rate_exchange[0]
        results_df['delta_s'] = delta_s
        results_df['delta_c'] = delta_c
        results_df['c_balance_fx'] = np.nancumsum(delta_c_fx) + c_balance
        results_df['s_supply'] = np.nancumsum(delta_s) + s_supply
        results_df['c_balance'] = np.nancumsum(delta_c) + c_balance
        results_df['bancor_minus_fx'] = results_df['bancor_price'] - results_df['fx_rate_exchange']
        results_df['contracted_s_tokens'] = -np.nancumsum(results_df['delta_s'])
        results_df['contracted_s_tokens_value'] = results_df['contracted_s_tokens'] * results_df['fx_rate_exchange']
        results_df['connector_value'] = results_df['c_balance']
        results_df['bancor_market_maker_value'] = results_df['contracted_s_tokens_value'] + results_df['c_balance']
        results_df['fx_market_maker_value'] = results_df['contracted_s_tokens_value'] + results_df['c_balance_fx']
        results_df['bancor_slippage_loss'] = results_df['fx_market_maker_value'] - results_df['bancor_market_maker_value']

        results_df.index.name = 'step'
        
        # remember all results
#        results_list.append(results_df)
        
        # remember total arbitrage losses
        total_arb_losses.append(results_df['bancor_slippage_loss'].iloc[-1])
        
    # update progress bar
    grid_run_progress.value += 1
    grid_run_progress.description = "{:.1%}".format(grid_run_progress.value/len(c_weight_grid))
    c_weight_arb_loss[c_weight] = np.mean(total_arb_losses)
    
# Show average arbitrage loss as function of connector weight 
arb_loss_df = pd.DataFrame(c_weight_arb_loss, index = ['avg_arb_loss']).T
arb_loss_df.index.name = 'c_weight'

iplot_dict = plotly_from_df(arb_loss_df, title='Average Arbitrage Loss (in Connector Token Units)', xlabel='c_weight', ylabel='avg_arb_loss', mode='lines', y_axis_type='log')
plotly.offline.iplot(iplot_dict)

IntProgress(value=0, description='0.0%', max=81)

# Uniswap

## Degrees of Freedom

Let $G_0$ denote the inital number of Celo Gold coins and $D_0$ the inital number of Celo Dollar coins in the respective tanks. The central equation for the uniswap system fixes the following relationship:

$G_0 \times D_0 = k = G_t \times D_t \quad \forall t$

where $k$ is the constant that follows from the initially chosen $G_0$ and $D_0$ quantities.

It can be shown that the price (for an infinitesimal small amount) of Celo Gold, to be paid in Celo Dollar units is simply

$P_t = \frac{D_t}{G_t}$.

**The path idenpendece property is satisfied be definition here.**

If a specific price has to be met when setting up both tanks, let's say $P_0=2$, then this already determines 

$\frac{D_0}{G_0}=2$ which implies  $D_0=2G_0$.

When setting up the tanks we can thus only freely choose one of the tank quantities if we want to satisfy a target initial exchange rate. $k$ is not a paramter that can be set on top of that - it just follows from setting $G_0$ and the $D_0$ that must be chosen to satisfy the given a target exchange rate $P_0$. 

**Note: For a constant k, there is only one degree of freedom given a target initial exchange rate! We will consider the initial Gold quantity as the single free parameter in the following sections.**

## What is affected by varying the inital Gold quantity and what isn't?

### Relative Percentage Changes in Tank Quantities

Define $x$ as the percentage change in $G$ and $y$ as percentage change in $D$ then

$G (1+x) \times D (1+y) = k $ 

implies that 

$y = -\frac{x}{1+x}$

which means that **the pct. change $y$ of $D$ that is implied by a pct. change $x$ of $G$ is independent of the quantities $G$ and $D$**. 



In [15]:
# show pct. change in D given x percent of change in G
from uniswap import *
x = np.linspace(-0.7,0.7,141)

y = []
for ix in x:    
    y.append(uniswap_get_y(ix))
    
pct_changes_df = pd.DataFrame(y, columns=['pct_change_dollars'], index=x)   

iplot_dict = plotly_from_df(pct_changes_df, title='Pct. Change in Dollars for Pct. Change in Gold', xlabel='pct_change_G (0.1 means 10%)', ylabel='pct_change_dollars (0.1 means 10%)', mode='lines', y_axis_type='linear')
plotly.offline.iplot(iplot_dict)

This is independent of the inital setup, i.e. independent of $G_0$, $D_0$ and $k$, as well as independent of the current price!

### Price Sensitivity

Let $z$ denote the percentage change in the price, then 

$P(1+z) = \frac{D(1+y)}{G(1+x)}$

and using the relationship of $y$ and $x$ from above gives

$z=\frac{1}{(1+x)^2}-1$

which shows that the **price sensitivity is also independent of the quantites of $D$ and $G$**. 

In [21]:
# show pct. change in P given x percent of change in G
x = np.linspace(-0.7,0.7,141)

z = []
for ix in x:    
    z.append(uniswap_get_z(ix))
    
pct_changes_price_df = pd.DataFrame(z, columns=['pct_change_price'], index=x)   

iplot_dict = plotly_from_df(pct_changes_price_df, title='Pct. Change in Price of Gold for Fractional Change in Gold Tank Through Dollar Transaction', xlabel='pct_change_G (0.1 means 10%)', ylabel='pct_change_P(0.1 means 10%)', mode='lines', y_axis_type='linear')
plotly.offline.iplot(iplot_dict)

The above relationship is independent of the level of $G$ and $D$!

### Pct. Price Change Given Absolute Change of G

The following formula from the last section

$z=\frac{1}{(1+x)^2}-1$

implies

$z=\frac{1}{(1+\frac{\Delta G}{G})^2}-1$. 

Thus the percentage change in the price given an absolute change of $\Delta G$ through trades vs. Celo Dollars depends on the level of $G$.

In [18]:
# show change delta_D (in number of coins) given change in G of delta_G coins
P = 2
G_grid = np.linspace(1000,10000,10) 
z_collected = pd.DataFrame()
for G in G_grid:
    D = P * G
    delta_G = np.linspace(-0.7,0.7,101)  * G

    z = []
    for i_delta_G in delta_G:    
        z.append(uniswap_get_z(i_delta_G/G))

    z = pd.DataFrame(z, columns=['Initial_G=' + str(G)], index=delta_G)   
    z_collected = pd.concat([z_collected, z], sort=False)
iplot_dict = plotly_from_df(z_collected, title='Pct. Price Change given Delta_G for Different Inital G', xlabel='delta_G (number of coins)', ylabel='pct_change_P(0.1 means 10%)', mode='lines', y_axis_type='linear')
plotly.offline.iplot(iplot_dict)

The curvature of the relationship depends on the initial level of $G$ and $D$! Here we assumed an inital price of $P=2$ which gives $D_0=2G_0$.

### Relative Changes in Number of Coins

Let us define $\Delta D$ as the change in the $C\$$ tank and $\Delta G$ as the change in the CG tank (not percentages now but number of coins). The initial Gold quantity determines the sensitivity of the price wrt to changes in the tank quantities. From the calculations in this [this document](https://github.com/runtimeverification/verified-smart-contracts/blob/uniswap/uniswap/x-y-k.pdf) in can be seen that

$\Delta D = -\frac{\Delta G}{G+\Delta G}  D$.

For a specific initial price P, this becomes
$\Delta D = -P\frac{\Delta G}{1+\frac{\Delta G}{G}}$.

In [19]:
# show change delta_D (in number of coins) given change in G of delta_G coins
P = 2
G_grid = np.linspace(1000,10000,10) 
delta_D_collected = pd.DataFrame()
for G in G_grid:
    D = P * G
    delta_G = np.linspace(-0.99,0.7,141)  * G

    delta_D = []
    for i_delta_G in delta_G:    
        delta_D.append(uniswap_get_deltaD(D, G, i_delta_G))

    delta_D = pd.DataFrame(delta_D, columns=['Initial_G=' + str(G)], index=delta_G)   
    delta_D_collected = pd.concat([delta_D_collected, delta_D], sort=False)
iplot_dict = plotly_from_df(delta_D_collected, title='Delta_D (in coins) given delta_G (in coins) for Different Inital G', xlabel='delta_G (number of coins)', ylabel='delta_D (number of coins)', mode='lines', y_axis_type='linear')
plotly.offline.iplot(iplot_dict)

The curvature of the relationship depends on the initial level of $G$ and $D$! Here we assumed an inital price of $P=2$ which gives $D_0=2G_0$.